# Adding OCO-[2|3]COGs

In [1]:
import boto3
import concurrent.futures
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
from osgeo import gdal

gdal.DontUseExceptions()

s3_client = boto3.client("s3")

In [3]:
source_bucket = "sdap-dev-zarr"
source_dir = "OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog"
# Set the number of worker processes
NUM_WORKERS = 4
destination_bucket = "veda-data-store-staging"
collection = "oco2-co2"
prefix = f"{source_dir}/{collection}"
collection_prefix_collection_id_map = {
    "oco3-co2": "oco3-co2-sam-l3-cogs",
    "oco2-co2": "oco2-co2-target-cogs",
    "oco3-sif": "oco3-co2-sif-cogs",
}

In [4]:
# List all objects in the source bucket
all_objects = []
continuation_token = None

while all_objects == [] or continuation_token is not None:
    args = dict(Bucket=source_bucket, Prefix=prefix)
    if continuation_token is not None:
        args["ContinuationToken"] = continuation_token
    response = s3_client.list_objects_v2(**args)
    [all_objects.append(obj["Key"]) for obj in response.get("Contents", [])]
    continuation_token = response.get("NextContinuationToken", None)

In [5]:
len(all_objects)

1149

In [6]:
all_objects[0]

'OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif'

In [9]:
def copy_object(object_key: str):
    copy_source = {"Bucket": source_bucket, "Key": object_key}
    destination_file_name = object_key.split("/")[-1]
    collection_prefix = destination_file_name.split("_")[0]
    destination_dir = collection_prefix_collection_id_map[collection_prefix]
    destination_object_key = f"{destination_dir}/{destination_file_name}"
    s3_client.copy_object(
        CopySource=copy_source, Bucket=destination_bucket, Key=destination_object_key
    )
    print(
        f"Copied: s3://{source_bucket}/{object_key} to s3://{destination_bucket}/{destination_object_key}"
    )


def process_file(object_key: str):
    """Function to process a single file."""
    dataset = gdal.Open(f"/vsis3/{source_bucket}/{object_key}", gdal.GA_ReadOnly)

    if dataset is None:
        return file_path, "Failed to open"

    band = dataset.GetRasterBand(1)
    stats = band.GetStatistics(True, True)

    # Check for errors
    err_type = gdal.GetLastErrorType()
    err_msg = gdal.GetLastErrorMsg()

    if "Failed to compute statistics, no valid pixels found in sampling" in err_msg:
        return dict(
            object_key=object_key,
            error_message=f"GDAL Error: {err_msg}",
            status="error",
        )
    else:
        copy_object(object_key=object_key)
        return dict(object_key=object_key, error_message=None, status="success")

## Test one file

In [10]:
first_object_key = all_objects[0]
process_file(first_object_key)

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif


Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif.aux.xml.


{'object_key': 'OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif',
 'error_message': None,
 'status': 'success'}

In [11]:
!aws s3 ls s3://veda-data-store-staging/oco2-co2-target-cogs/ #| wc -l

2025-03-27 21:52:40      82031 oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif


In [12]:
def main():
    failures = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        futures = {
            executor.submit(process_file, object_key): object_key
            for object_key in all_objects[0:100]
        }

        for idx, future in enumerate(as_completed(futures)):
            if idx % 100 == 0:
                print(f"processed file {idx}")
            result = future.result()
            if result["status"] == "error":
                object_key = result["object_key"]
                failures.append((object_key, result))
                # print(f"{object_key}: {result['error_message']}")

    # Save results to a log file
    with open("failed_files.log", "w") as f:
        for object_key, result in failures:
            f.write(f"{object_key}\n")


main()

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044043Z_filtered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_anmyeondoKo_2016-03-25T044043Z_filtered_xco2.tifCopied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-05-19T044701Z_filtered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_anmyeondoKo_2016-05-19T044701Z_filtered_xco2.tif
processed file 0
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-05-19T044658Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_anmyeo

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044043Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-05-19T044701Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-03-25T044042Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2016-05-19T044658Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2017-02-15T044700Z_unfiltered_xco2.tif.aux.xml.


Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2020-03-18T045244Z_filtered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_anmyeondoKo_2020-03-18T045244Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2014-09-26T143542Z_filtered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_ascensionIsland_2014-09-26T143542Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2014-09-26T143441Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_ascensionIsland_2014-09-26T143441Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2020-03-18T045243Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_anmyeon

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2020-03-18T045244Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2014-09-26T143542Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2014-09-26T143441Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_anmyeondoKo_2020-03-18T045243Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2015-01-16T143300Z_unfiltered_xco2.tif.a

Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2015-02-08T144117Z_filtered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_ascensionIsland_2015-02-08T144117Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2016-11-09T144213Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_ascensionIsland_2016-11-09T144213Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2016-10-15T144617Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_ascensionIsland_2016-10-15T144617Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_ascensionIsland_2016-12-20T143344Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/

...

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2014-10-30T210714Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-01-02T210740Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-01-02T210740Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-01-27T210135Z_filtered_xco2.tif.aux.xml.


Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-03-07T210821Z_filtered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_caltechCA_2015-03-07T210821Z_filtered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-01-27T210135Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_caltechCA_2015-01-27T210135Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-03-07T210821Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_caltechCA_2015-03-07T210821Z_unfiltered_xco2.tif
Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-08-05T211508Z_filtered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_caltechCA_2015-08-05T211508Z_filter

Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-03-07T210821Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-01-27T210135Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-03-07T210821Z_unfiltered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-08-05T211508Z_filtered_xco2.tif.aux.xml.
Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-07-04T211858Z_filtered_xco2.tif.aux.xml.


Copied: s3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-07-04T211527Z_unfiltered_xco2.tif to s3://veda-data-store-staging/oco2-co2-target-cogs/oco2-co2_caltechCA_2015-07-04T211527Z_unfiltered_xco2.tif


Warning 1: Unable to save auxiliary information in /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco2-co2_caltechCA_2015-07-04T211527Z_unfiltered_xco2.tif.aux.xml.
